In [26]:
import sqlite3
import csv
import pandas as pd

In [27]:
# read data

data_df = pd.read_csv("data/bank-additional-full.csv", sep=";",header=0)
data_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [28]:
data_job = data_df["job"].unique()

##### Convert Job column to table

In [29]:
list_job = [(iterator,_job) for iterator,_job in enumerate(data_job)]
table_job = pd.DataFrame(columns=["Id","Job"],data=list_job)
table_job.head()

,Id,Job
0,0,housemaid
1,1,services
2,2,admin.
3,3,blue-collar
4,4,technician


#### Convert marital to table

In [30]:
data_marital = data_df["marital"].unique()
data_marital = [(i,_marital) for i,_marital in enumerate(data_marital)]
table_marital = pd.DataFrame(columns=["Id","Marital"], data=data_marital)
table_marital.head()

,Id,Marital
0,0,married
1,1,single
2,2,divorced
3,3,unknown


In [31]:
data_df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [32]:
data = data_df.copy()

#### Reemplazo de valores

In [33]:
job_map = dict(zip(table_job["Job"], table_job["Id"]))

if "job" in data.columns:
    data["job_id"] = data["job"].map(job_map)
    data = data.drop(columns=["job"],errors="ignore")
data.head()


,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,job_id
0,56,married,basic.4y,no,no,no,telephone,may,mon,261,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
1,57,married,high.school,unknown,no,no,telephone,may,mon,149,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1
2,37,married,high.school,no,yes,no,telephone,may,mon,226,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1
3,40,married,basic.6y,no,no,no,telephone,may,mon,151,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2
4,56,married,high.school,no,no,yes,telephone,may,mon,307,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1


In [34]:
job_map_marital = dict(zip(table_marital["Marital"], table_marital["Id"]))

if "marital" in data.columns:
    data["marital_id"] = data["marital"].map(job_map_marital)
    data = data.drop(columns=["marital"],errors="ignore")
data.head()

,age,education,default,housing,loan,contact,month,day_of_week,duration,campaign,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,job_id,marital_id
0,56,basic.4y,no,no,no,telephone,may,mon,261,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0,0
1,57,high.school,unknown,no,no,telephone,may,mon,149,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1,0
2,37,high.school,no,yes,no,telephone,may,mon,226,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1,0
3,40,basic.6y,no,no,no,telephone,may,mon,151,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2,0
4,56,high.school,no,no,yes,telephone,may,mon,307,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1,0


#### Conexión a sql mysql

conectar a db mysql desde python

In [35]:
data.columns

Index(['age', 'education', 'default', 'housing', 'loan', 'contact', 'month',
       'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'job_id', 'marital_id'],
      dtype='object')

In [36]:
import mysql.connector

# Conexión a MySQL por IP
conn = mysql.connector.connect(
    host="127.0.0.1",   
    user="root",
    password="root",
    database="banco_datadb",
    port=3306               
)

In [37]:
cursor = conn.cursor()

In [ ]:
for _, fila in data.iterrows():
    sql = """
    INSERT INTO TABLE_DATA_BANK (
        age, education, default, housing, loan, contact, month,
        day_of_week, duration, campaign, pdays, previous, poutcome, euribor3m,
        nr.employed, y, job_id, marital_id
    )
    VALUES (
        %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s,
        %s, %s, %s, %s
    )
    """

    valores = (
        fila["age"], fila["education"], fila["default"], fila["housing"], fila["loan"],
        fila["contact"], fila["month"], fila["day_of_week"], fila["duration"],
        fila["campaign"], fila["pdays"], fila["previous"], fila["poutcome"], fila["euribor3m"],
        fila["nr.employed"], fila["y"], fila["job_id"], fila["marital_id"]
    )

    cursor.execute(sql, valores)

conn.commit()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'default, housing, loan, contact, month,
        day_of_week, duration, campaign,' at line 2

In [ ]:
1 and 1

1